<a href="https://colab.research.google.com/github/AMS-L/NLP-LAB7/blob/main/nlp_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP - LAB 7


In [2]:
!pip install transformers
!pip install datasets
!pip install transformers==4.28.0
!pip install --upgrade accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 92.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00

In [3]:
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from transformers import DataCollatorWithPadding

## Sentence Classification

### 1. Fine-tune the model on the training data. 

In [4]:
dataset = load_dataset("imdb")
train_dataset, test_dataset = dataset['train'], dataset['test']

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
    
train_dataset, test_dataset = dataset['train'], dataset['test']
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
# Training arguments
args = TrainingArguments(
    "test-imdb",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)


In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.we

In [9]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).astype(np.float32).mean().item()}


In [10]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.223500,0.193192,0.926920


TrainOutput(global_step=1563, training_loss=0.25222084816647733, metrics={'train_runtime': 1619.0506, 'train_samples_per_second': 15.441, 'train_steps_per_second': 0.965, 'total_flos': 3311684966400000.0, 'train_loss': 0.25222084816647733, 'epoch': 1.0})

### 3. Evaluate the model in term of accuracy on the test data.

In [11]:
trainer.evaluate()

{'eval_loss': 0.19319181144237518,
 'eval_accuracy': 0.9269199967384338,
 'eval_runtime': 434.5609,
 'eval_samples_per_second': 57.529,
 'eval_steps_per_second': 3.597,
 'epoch': 1.0}

On a ici une accuracy de 92,7 %.

In [12]:
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)

actual_labels = predictions.label_ids

 ### 4. For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

In [13]:
misclassified_indices = np.where(predicted_labels != actual_labels)[0]

for i in misclassified_indices[:10]:
    print(f"Text: {test_dataset['text'][i]}")
    print(f"Predicted label: {predicted_labels[i]}")
    print(f"Actual label: {actual_labels[i]}\n")

Text: First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!
Predicted label: 1
Actual label: 0

Text: Ben, (Rupert Grint), is a deeply unhappy adolescent, the son of his unhappily married parents. His father, (Nicholas Farrell), is a vicar and his mother, (Laura Linney), is ... well, let's just say she's a somewhat hypocritical soldier in Jesus' army. It's only when he takes a s

On peut voir dans ce contexte que les deux commentaires sont négatifs mais qu'ils ressortent positifs dans la classification. Cela pourrait être du au fait que la logique de base de la transformer consiste à utiliser des mécanismes d'attention pour calculer des représentations contextuelles des mots ou des tokens dans une séquence. 
Ces représentations contextuelles capturent les informations contextuelles à la fois locales et globales, ce qui permet au modèle de comprendre les relations et les dépendances entre les éléments de la séquence. 

Dans ce contexte, on voit que le premier commentaire établit beaucoup de formulations contextuelles négatives mais dit qu'il aime l'acteur ce qui fait que globalement il ya beaucou de formulation contextuelle positive mais ne parlant pas du film, et une énumération des défauts du film cité ce qui peut porter à confusion le transformer.

Le deuxième admet beaucoup d'adjectifs négatifs mais la nuance entre la description négatives des personnage dans un contexte fictif et le commentaire sur le film en lui même peut induire le transformer en erreur.

### 5. What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course? And compared to a recurrent model like an RNN or an LSTM?

#### -> Les avantages :

  Le modèle de transformer comme DistilBERT est capable de capturer des relations complexes entre les mots ou les tokens d'une séquence grâce à son architecture d'attention. Contrairement à Naive Bayes, qui suppose une indépendance entre les caractéristiques, DistilBERT peut modéliser les dépendances à longue distance, ce qui lui permet de saisir des informations contextuelles plus riches et de mieux comprendre le langage naturel.

  Dans un second temps, DistilBERT apprend aussi des représentations de mots ou de tokens directement à partir des données, ce qui lui permet d'adapter ces représentations à la tâche spécifique. En revanche, Naive Bayes utilise des statistiques de fréquence pour estimer les probabilités, ce qui peut être moins adaptatif et moins précis dans des cas où les caractéristiques sont complexes et fortement interdépendantes.

  Pour finir, DistilBERT intègre une compréhension sémantique des mots ou des tokens grâce à l'utilisation d'embeddings, qui représentent les mots dans un espace vectoriel continu. Cela permet au modèle de comprendre les similitudes et les relations entre les mots, ce qui peut être utile pour des tâches telles que la recherche de similarités ou la génération de texte cohérent.

#### -> Les inconvénient :

  DistilBERT, est plus complexes et nécessite des ressources computationnelles plus importantes que les modèles Naive Bayes. Ils peuvent être plus lents à l'entraînement et à l'inférence, ce qui peut être un inconvénient dans des scénarios où le temps de réponse est important.

  Les modèles de transformer ont souvent besoin d'un volume plus important de données d'entraînement pour atteindre de bonnes performances. Par conséquent, ils peuvent nécessiter un ensemble de données plus large et plus diversifié par rapport à Naive Bayes, qui est moins exigeant en termes de données d'entraînement.

#### -> Comparaison avec les modèles récurrents (RNN ou LSTM) :

  Les modèles récurrents sont conçus pour capturer les dépendances séquentielles à long terme dans une séquence. Cela peut être avantageux pour les tâches où l'ordre des éléments est important, comme la traduction automatique ou la génération de texte cohérent. DistilBERT, en revanche, n'a pas d'architecture récurrente et dépend entièrement des mécanismes d'attention pour capturer les dépendances.

  Par la suite, DistilBERT, peut prendre en compte le contexte global de la séquence grâce à l'attention sur tous les tokens. Cela lui permet de saisir les relations non seulement à l'intérieur d'une fenêtre de contexte fixe, mais aussi à travers toute la séquence. Les modèles récurrents ont quant à eux une mémoire limitée et peuvent avoir du mal à capter les dépendances à très long terme.
  
  Pour finir, DistilBERT bénéficie  d'un préentraînement sur de vastes ensembles de données non supervisées, ce qui lui permet d'apprendre des représentations linguistiques générales. 
  Cela peut être bénéfique pour des tâches spécifiques avec des ensembles de données plus petits. Les modèles récurrents, en revanche, nécessitent généralement un entraînement plus spécifique à la tâche.

### 6. The model only accepts inputs of maximum 512 tokens. Propose and implement a solution that goes around that. 